In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

import pandas as pd

In [2]:
wine = load_wine()
wine_data = wine.data[0:130]
wine_target = wine.target[0:130]

In [3]:
train_X, test_X, train_Y, test_Y = train_test_split(wine_data, 
                                                    wine_target, 
                                                    test_size=0.2)

In [4]:
# 데이터를 텐서 형태로 변환
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).float()

train = TensorDataset(train_X, train_Y)
train_loader = DataLoader(train, batch_size=16, shuffle=True)

In [5]:
# 신경망 구성
class Net(nn.Module):
    
    def __init__(self):
        
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 96)
        self.fc2 = nn.Linear(96, 96)
        self.fc3 = nn.Linear(96, 96)
        self.fc4 = nn.Linear(96, 96)
        self.fc5 = nn.Linear(96, 96)
        self.fc6 = nn.Linear(96, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x)) 
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))

        x = self.fc6(x)
        return F.log_softmax(x, dim=1)


model = Net()

In [6]:
#오차함수
criterion = nn.CrossEntropyLoss()

In [7]:
#최적화 담당
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [8]:
#학습 300회 ㄱㄱ 씽
for epoch in range(300):
    
    # 누적 오차를 담당할 변수
    total_loss = 0
  
    # 아까 만들어놓은 미니 배치에서 각각 변수를 꺼내온다.
    for train_x, train_y in train_loader:
        # 각각의 값을 변수로 만든다.
        train_x, train_y = Variable(train_x), Variable(train_y)

        # 경사 초기화
        optimizer.zero_grad()

        # 순전파
        output = model(train_x)

        # 오차계산. ouutput 과 train_y 비교
        loss = criterion(output, train_y)

        # 역전파 계산
        loss.backward()

        # 가중치 업데이트. learning rate만큼 
        optimizer.step()

        #총 오차 업데이트
        total_loss += loss.data.item()

    # 10 회 마다 현재 오차를 출력
    if (epoch + 1) % 10 == 0:
        print(epoch + 1, total_loss)

10 4.1218357384204865
20 3.138661891222
30 3.64332115650177
40 2.7002440989017487
50 3.593239277601242
60 5.087172329425812
70 2.7422373294830322
80 3.0029089152812958
90 2.287327215075493
100 2.046235054731369
110 2.5580506324768066
120 1.8554191291332245
130 1.7711554169654846
140 1.5086041912436485
150 1.784321814775467
160 1.5675443708896637
170 1.703890711069107
180 1.5514112412929535
190 1.5075144469738007
200 1.8952373713254929
210 1.7592565417289734
220 1.9253981783986092
230 1.401431068778038
240 1.6575833186507225
250 1.4886344447731972
260 1.56826601177454
270 1.387528896331787
280 1.5204885005950928
290 1.4569556415081024
300 1.5496127530932426


In [9]:
# 테스트 데이터를 집어 넣는다.
test_x, test_y = Variable(test_X), Variable(test_Y)
# 테스트 데이터를 집어 넣어서 학습 시킨 다음, max 값(확률이 더 높은 값)을 출력한다.
result = torch.max(model(test_x).data, 1)[1]
# 정확도 계산
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())
print("accuracy= ",accuracy)

accuracy=  0.9615384615384616
